In [0]:
import torch as t
import numpy as np
from torch import nn
from torchvision.datasets import CIFAR10

In [0]:
class Alexnet(nn.Module):
    def __init__(self):
        super(Alexnet, self).__init__()
        
        # nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0,
        # dilation=1, groups=1, bias=True, padding_mode='zeros',)
        self.conv1 = nn.Sequential(nn.Conv2d(3, 64, 5), nn.ReLU(True))
        self.maxpool1 = nn.MaxPool2d(3, 2)
        self.conv2 = nn.Sequential(nn.Conv2d(64, 64, 5, 1), nn.ReLU(True))
        self.maxpool2 = nn.MaxPool2d(3, 2)
        self.fc1 = nn.Sequential(nn.Linear(1024, 384), nn.ReLU(True))
        self.fc2 = nn.Sequential(nn.Linear(384, 192), nn.ReLU(True))
        self.fc3 = nn.Linear(192, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        # 拉平矩阵
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x

In [3]:
alexnet = Alexnet()
alexnet

Alexnet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Sequential(
    (0): Linear(in_features=1024, out_features=384, bias=True)
    (1): ReLU(inplace)
  )
  (fc2): Sequential(
    (0): Linear(in_features=384, out_features=192, bias=True)
    (1): ReLU(inplace)
  )
  (fc3): Linear(in_features=192, out_features=10, bias=True)
)

In [4]:
test_input = t.zeros((1, 3, 32, 32), requires_grad=True)
out = alexnet(test_input)
out.shape

torch.Size([1, 10])

In [5]:
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 标准化，这个技巧之后会讲到
    x = x.transpose((2, 0, 1)) # 将 channel 放到第一维，只是 pytorch 要求的输入方式
    x = t.from_numpy(x)
    return x
     
train_set = CIFAR10('./data', train=True, transform=data_tf, download=True)
train_data = t.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = CIFAR10('./data', train=False, transform=data_tf, download=True)
test_data = t.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

device = t.device('cuda')
net = Alexnet().to(device)
optimizer = t.optim.SGD(net.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss().to(device)

0it [00:00, ?it/s]

170500096it [00:09, 17806953.19it/s]                               


Files already downloaded and verified


In [6]:
from train_util import train

train(net, train_data, test_data, 20, optimizer, criterion)

Epoch [0]
Train loss: 1331.659947, Train acc: 0.250
Test loss: 127.298236, Test acc: 0.438
Time 00:00:08
-------------------------------------------------
Epoch [1]
Train loss: 983.936461, Train acc: 0.500
Test loss: 108.496559, Test acc: 0.500
Time 00:00:16
-------------------------------------------------
Epoch [2]
Train loss: 816.956243, Train acc: 0.812
Test loss: 84.424404, Test acc: 0.875
Time 00:00:24
-------------------------------------------------
Epoch [3]
Train loss: 694.969443, Train acc: 0.625
Test loss: 86.515812, Test acc: 0.750
Time 00:00:32
-------------------------------------------------
Epoch [4]
Train loss: 600.708918, Train acc: 0.812
Test loss: 81.601642, Test acc: 0.688
Time 00:00:40
-------------------------------------------------
Epoch [5]
Train loss: 522.032478, Train acc: 0.750
Test loss: 86.126449, Test acc: 0.375
Time 00:00:48
-------------------------------------------------
Epoch [6]
Train loss: 457.999559, Train acc: 0.812
Test loss: 72.813909, Test a